## 4. Modell Training und Auswahl

Die CSV-Datei enthält die kodierten Daten, die Sie für das Training und die Vorhersage verwenden werden.
Die joblib-Datei enthält die Encoder, die Sie zum Decodieren der kategorischen Merkmale benötigen, wenn Sie die Originalkategorien wiederherstellen möchten.


In [ ]:
from joblib import load
import pandas as pd

# Laden der kodierten Daten aus der CSV-Datei
encoded_data_path = '../model/encoded_data.csv'
encoded_data = pd.read_csv(encoded_data_path)


# Laden der Encoder aus der joblib-Datei
encoders_path = '../model/encoders.joblib'
loaded_encoders = load(encoders_path)

# Zeigen Sie die ersten paar Zeilen der kodierten Daten an
encoded_data.head(), loaded_encoders


## XGBoost

In [ ]:
from sklearn.model_selection import train_test_split

# Trennung der Merkmale und Labels
X = encoded_data.drop(columns=['Preis (€)'])
y = encoded_data['Preis (€)']

# Aufteilen der Daten in Trainings- und Testsets (80% Training, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Größe der Trainings- und Testsets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


### Trainieren des Modells

In [ ]:
from xgboost import XGBRegressor

# Erstellen des XGBoost-Regressionsmodells
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Training des Modells mit den Trainingsdaten
xgb_model.fit(X_train, y_train)


### Bewertung des Modells

In [ ]:
from sklearn.metrics import mean_absolute_error

# Vorhersagen auf den Testdaten
y_pred = xgb_model.predict(X_test)

# Berechnen des mittleren absoluten Fehlers (MAE) zur Beurteilung der Modellleistung
mae = mean_absolute_error(y_test, y_pred)
print("Mittlerer absoluter Fehler (MAE):", mae)

In [ ]:
print("Mögliche Werte für 'Marke':", loaded_encoders['Marke'].classes_)
print("Mögliche Werte für 'Modell':", loaded_encoders['Modell'].classes_)

### Benutzereingabe für die Vorhersage akzeptieren:

In [ ]:
def get_user_input(encoders):
    user_input = {}
    for feature, encoder in encoders.items():
        print(f"Bitte wählen Sie einen Wert für {feature} aus den folgenden Optionen:")
        classes_str = ", ".join(map(str, encoder.classes_))  # Formatierung der Klassen als Zeichenketten
        print(classes_str)
        value = input(f"Geben Sie den Wert für {feature} ein: ")
        if feature == 'Modell':
            value = ' ' + value  # Leerzeichen voranstellen, wenn es sich um das Merkmal "Modell" handelt
        while value not in encoder.classes_:
            print("Ungültiger Wert. Bitte versuchen Sie es erneut.")
            value = input(f"Geben Sie den Wert für {feature} ein: ")
            if feature == 'Modell':
                value = ' ' + value
        user_input[feature] = value
    # Hier können Sie weitere numerische Eingaben hinzufügen
    user_input['Baujahr'] = int(input("Geben Sie das Baujahr ein: "))
    user_input['Kilometerstand'] = int(input("Geben Sie den Kilometerstand ein: "))
    user_input['Verbrauch (l/100km)'] = float(input("Geben Sie den Verbrauch (l/100km) ein: "))
    user_input['Motorgröße (l)'] = float(input("Geben Sie die Motorgröße (l) ein: "))
    return user_input

In [ ]:
# Benutzereingabe erhalten
user_input = get_user_input(loaded_encoders)

# Vorhersage mit der Funktion von zuvor
predicted_price = predict_price(user_input, xgb_model, loaded_encoders)

# Ausgabe der vorhergesagten Preisvorhersage
print("Vorhergesagter Preis (€):", predicted_price)
